In [1]:
# setting up the workplace
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# nltk.download('stopwords')
from functions import description_cleaner, vocabulary_creator, reverse_index_creator
from engine import non_ranked_engine

restaurants_df = pd.read_csv("restaurants_i.tsv", sep="\t", header = 0)

In [2]:
# Main preprocessing

restournats_descriptions = restaurants_df.description

parsed_descriptions = description_cleaner(restournats_descriptions)

ID_descritpion, vocabulary, tot_n_of_words = vocabulary_creator(parsed_descriptions)

reverse_index = reverse_index_creator(ID_descritpion)

pd.Series(vocabulary).to_csv("vocabulary.csv", index=True, encoding="utf-8", header= False)



In [3]:
# NON ranked serch engine
done = False
sample_input = "modern seasonal cuisine"
while (not done):
    # sample_input = input("what do you want to eat?\n")
    done = non_ranked_engine(sample_input, restaurants_df, vocabulary, reverse_index)

    

We found 41 matches!

╭─────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────────╮
│ Restaurant Name         │ Address                   │ Description               │ Website                   │
├─────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┤
│ Casin del Gamba         │ via Roccolo Pizzati 1     │ The journey to get here – │ https://www.casindelgamba │
│                         │                           │ a winding road throug...  │ .it/                      │
├─────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┤
│ San Giorgio             │ viale Brigate Bisagno 69r │ Situated in the city      │ https://www.ristorantesan │
│                         │                           │ albeit not right in the   │ giorgiogenova.it/         │
│                         │                           │ ce...                     